In [ ]:
import os
import sys
import datacube
import numpy as np
import pandas as pd
import xarray as xr
from datacube.utils import geometry
from datacube.utils.geometry import CRS
from datacube import Datacube
import geopandas as gpd
import affine


import rasterio
import matplotlib.pyplot as plt
import rasterio.features

# Import external functions from dea-notebooks using relative link to Scripts
sys.path.append('../10_Scripts')
import DEAPlotting
# import DEADataHandling
# import SpatialTools

# Connect to datacube database
dc = datacube.Datacube(app='Time series animation')

%load_ext autoreload
%autoreload 2


def interpolate_timeseries(ds, freq='7D', method='linear'):
    
    """
    Interpolate new data between each existing xarray timestep at a given
    frequency. For example, `freq='7D'` will interpolate new values at weekly
    intervals from the start time of the xarray dataset to the end time. 
    `freq='24H'` will interpolate new values for each day, etc.
    
    Parameters
    ----------  
    ds : xarray Dataset
        The xarray dataset to interpolate new time-step observations for.
        
    freq : string or int, optional
        An optional string or integer giving the frequency at which to interpolate new 
        time-step observations. To interpolate a time series at a constant/regualar time 
        frequency (e.g. weekly or monthly), use a Pandas offset alias string (e.g. '7D'): 
        https://pandas.pydata.org/pandas-docs/stable/timeseries.html#timeseries-offset-aliases
        Alternatively, specify an integer to insert X evenly spaced new time steps between
        each existing timestep in the dataset (e.g. a value of 1 will create 1 new frame
        between every existing timestep). Defaults to Pandas offset '7D' which interpolates 
        new values at weekly intervals.
        
    method : string, optional
        An optional string giving the interpolation method to use to generate new time-steps.
        Default is 'linear'; options are {'linear', 'nearest'} for multidimensional arrays and
        {'linear', 'nearest', 'zero', 'slinear', 'quadratic', 'cubic'} for 1-dimensional arrays.
        
    Returns
    -------
    A xarray dataset covering the same time period as `ds`, but with new timesteps interpolated 
    for each time-step given by `freq`.
        
    """    
    
    if isinstance(freq, str):
    
        # Use pandas to generate dates from start to end of ds at a given frequency
        start_time = ds.isel(time=0).time.values.item() 
        end_time = ds.isel(time=-1).time.values.item()    
        from_to = pd.date_range(start=start_time, end=end_time, freq=freq)

        # Use these dates to linearly interpolate new data for each new date
        print('Interpolating {} time-steps at {} intervals'.format(len(from_to), freq))
        return ds.interp(coords={'time': from_to}, method=method)
    
    elif isinstance(freq, int):
        
        # Get array of all timesteps
        ds_times = ds.time.values

        # Create list to save output
        interp_times_all = []

        # For each pair of timestamps, interpolate intermediate frames
        for i, value in enumerate(ds_times[:-1]):

            # Interpolate new dates between start and end dates
            interp_times = pd.date_range(start=value, end=ds_times[i+1], periods=(2 + freq))

            # Keep only new dates, not start and end dates
            interp_times_all.append(interp_times[1:-1].values)

        # Combine new dates with old dates and sort
        from_to = np.concatenate([*interp_times_all, ds_times])
        from_to.sort()
        
        # Use these dates to linearly interpolate new data for each new date
        print('Interpolating {} time-steps by generating {} intermediate frames'.format(len(from_to), freq))
        return ds.interp(coords={'time': from_to}, method=method)


In [4]:
datacube.__version__

'1.6.2+398.g0e94625d'

In [ ]:
import geopandas as gpd
import os

nsw_boundary = gpd.read_file('/g/data/r78/rt1527/shapefiles/australia/new_south_wales/cstnswcd_r.shp')
# nsw_boundary = gpd.read_file('/g/data/r78/rt1527/dea-notebooks/Animations/state_subset.shp')
nsw_boundary = nsw_boundary[nsw_boundary.FEAT_CODE == 'mainland']
nsw_boundary = nsw_boundary.to_crs({'init': 'EPSG:3577'}).unary_union
nsw_boundary

In [ ]:
# Use the polygon to create a datacube geometry object based on geojson and projection
geom = geometry.Geometry(nsw_boundary.__geo_interface__, 
                         crs=geometry.CRS('EPSG:3577'))

# Use this geometry to create a spatiotemporal query that will be passed on to the datacube
query = {'geopolygon': geom,   
         'time': ('2011', '2011'),
         'output_crs': 'EPSG:3577',
         'resampling': 'average',
         'resolution': (-2500, 2500)} 

# wofs_aproct_ds = dc.load(product='wofs_apr_oct_summary', measurements=['frequency'], dask_chunks={'time': 1}, **query)
# wofs_novmar_ds = dc.load(product='wofs_nov_mar_summary', measurements=['frequency'], dask_chunks={'time': 1}, **query)
# wofs_ds = xr.concat([wofs_aproct_ds, wofs_novmar_ds], dim='time').sortby('time')

wofs_ds = dc.load(product='wofs_annual_summary', measurements=['frequency'], dask_chunks={'time': 1}, **query)
# wofs_resamp = wofs_ds.where(wofs_ds.frequency > 0).coarsen(x=10, y=10, boundary='trim').mean(keep_attrs=True)
# wofs_resamp.isel(time=0).frequency.plot(size=6)

In [ ]:
wofs_ds.isel(time=0).frequency.plot(size=6)

In [ ]:
mask = rasterio.features.rasterize(shapes=[(nsw_boundary, 1)],
                                   out_shape=(wofs_resamp.dims['y'], wofs_resamp.dims['x']),
                                   transform=wofs_ds.geobox.transform * affine.Affine.scale(10))

plt.imshow(mask)

In [ ]:
wofs_resamp = wofs_resamp.where(mask).compute()


In [ ]:
wofs_animation_ds = interpolate_timeseries(wofs_resamp, freq=10, method='linear')

In [ ]:
DEAPlotting.animated_timeseries(ds=wofs_animation_ds,
                                output_path=f'animated_timeseries_NSWWOfS_2.mp4',
                                bands=['frequency'],
                                interval=80,
                                width_pixels=800,
                                percentile_stretch=[0.0, 0.99],
                                onebandplot_cbar=False,
                                show_date=False,
                                title=wofs_animation_ds.time.dt.year.values.tolist(),
                                annotation_kwargs={'xy': (0.15, 0.13)},
                                onebandplot_kwargs={'interpolation': 'nearest', 'cmap': 'Blues', 'vmin': 0.0, 'vmax': 0.3}  # 
                               )

In [ ]:
wofs_animation_ds